# Neural Network


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error

2024-11-12 17:57:56.060499: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 17:57:56.074232: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 17:57:56.077950: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 17:57:56.088510: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-12 17:57:56.811429: W tensorflow/compiler/tf2

### Hiperparametros y puntos de mejora
- Las variables del data set que se tienen en cuenta (modelo, año_modelo, millaje, motor, transmision, sin_daños)
- El Encoder the variables categoricas (LabelEncoder)
- El Scaler (Standard)
- La funcion de activacion (ReLu)
- La topologia de la red (6-64-32-1)
- El optimizador (ADAM)
- La funcion de error (Mean Squared Error) 
- La cantidad de epocas (10)
- El batch size (32)

In [2]:
# Load the training and test data
train_data_path = '../resources/train.csv'

df_train = pd.read_csv(train_data_path)

In [3]:
columns_to_drop = ['id', 'numero', 'color_exterior', 'color_interior', 'tipo_combustible', 'accidente', 'marca']
df_train = df_train.drop(columns=[col for col in columns_to_drop if col in df_train.columns])

df_train['sin_daños'] = df_train['sin_daños'].fillna("Unknown")

# Separate features and target
X = df_train.drop(columns=['precio'])
y = df_train['precio']

# Encode categorical features
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Scale numeric features
scaler = StandardScaler()
X[X.select_dtypes(include=['int64', 'float64']).columns] = scaler.fit_transform(X.select_dtypes(include=['int64', 'float64']))

# Define KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize model performance lists
mae_scores = []

# Cross-validation loop
for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Build the model
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, validation_data=(X_val, y_val))

    # Predict and evaluate
    y_pred = model.predict(X_val).flatten()
    mae = mean_absolute_error(y_val, y_pred)
    mae_scores.append(mae)
    print(f"Fold MAE: {mae}")

# Report the cross-validated mean MAE
print(f"\nCross-validated Mean MAE: {np.mean(mae_scores):.4f}")

/home/joaquin/Desktop/nlp/nlp/nlpenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-12 17:57:57.688471: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-11-12 17:57:57.688496: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: JGirod
2024-11-12 17:57:57.688504: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: JGirod
2024-11-12 17:57:57.688598: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 535.183.1
2024-11-12 17:57:57.688618: I external/local_xla/xla/stream_executor/cuda/cuda_diag

1061/1061 ━━━━━━━━━━━━━━━━━━━━ 0s 404us/step
Fold MAE: 20419.2803569193


/home/joaquin/Desktop/nlp/nlp/nlpenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1061/1061 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step
Fold MAE: 21144.254319798434


/home/joaquin/Desktop/nlp/nlp/nlpenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1061/1061 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step
Fold MAE: 21526.536010965207


/home/joaquin/Desktop/nlp/nlp/nlpenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1061/1061 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step
Fold MAE: 21501.34726747282


/home/joaquin/Desktop/nlp/nlp/nlpenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1061/1061 ━━━━━━━━━━━━━━━━━━━━ 1s 446us/step
Fold MAE: 20924.091789051134

Cross-validated Mean MAE: 21103.1019


In [4]:
# Load the guess.csv
guess_df = pd.read_csv('../resources/guess.csv')

# Drop the same columns as in training
columns_to_drop = ['numero', 'color_exterior', 'color_interior', 'tipo_combustible', 'accidente', 'marca', 'precio']
guess_df = guess_df.drop(columns=[col for col in columns_to_drop if col in guess_df.columns])

guess_df['sin_daños'] = guess_df['sin_daños'].fillna("Unknown")

# Separate the 'id' column to use it later in the output
id_column = guess_df['id']
guess_df = guess_df.drop(columns=['id'])

# Encode categorical features in the guess data
for col in guess_df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    guess_df[col] = le.fit_transform(guess_df[col])

# Scale numeric features (same scaling used for training)
scaler = StandardScaler()
guess_df[guess_df.select_dtypes(include=['int64', 'float64']).columns] = scaler.fit_transform(guess_df.select_dtypes(include=['int64', 'float64']))

# Make predictions
predictions = model.predict(guess_df).flatten()

# Create a DataFrame with 'id' and 'prediction' columns
output_df = pd.DataFrame({
    'id': id_column,
    'prediction': predictions
})

# Save to CSV
output_df.to_csv('../output/predictions.csv', index=False)

print("Predictions have been saved to 'predictions.csv'")


590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step
Predictions have been saved to 'predictions.csv'
